In [1]:
# Instalar si no se tiene el paquete, de lo contrario comentar
import Pkg; Pkg.add("Cbc")

┌ Warning: attempting to remove probably stale pidfile
│   path = "/Users/antoarriagada/.julia/registries/.pid"
└ @ FileWatching.Pidfile ~/.julia/juliaup/julia-1.10.2+0.x64.apple.darwin14/share/julia/stdlib/v1.10/FileWatching/src/pidfile.jl:273
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Clp_jll ──────── v100.1700.700+1
   Installed OpenBLAS32_jll ─ v0.3.24+0
   Installed Cgl_jll ──────── v0.6000.600+0
   Installed Cbc_jll ──────── v200.1000.800+0
   Installed CoinUtils_jll ── v200.1100.600+0
   Installed Osi_jll ──────── v0.10800.700+0
   Installed Cbc ──────────── v1.2.0
   Installed ASL_jll ──────── v0.1.3+0
   Installed METIS_jll ────── v5.1.2+0
   Installed MUMPS_seq_jll ── v500.600.201+0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [9961bab8] + Cbc v1.2.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [9961bab8] + Cbc v1.2.0
  [ae81ac8f] + ASL_jll v0.1.3+0
  [38041ee0] + Cbc_jll v200.1000.800+0

In [1]:
using JuMP
using GLPK 
using Cbc

In [2]:
# Función que permite leer las instancias de bin packing desde un archivo
function read_bin_packing_instance(filename)
    problems = []
    open(filename, "r") do file
        lines = readlines(file)
        P = parse(Int, lines[1])  # Número de instancias de problemas
        index = 2
        for _ in 1:P
            problem_id = strip(lines[index])
            index += 1
            bin_capacity, num_items, best_known_solution = split(lines[index])
            bin_capacity = parse(Float64, bin_capacity)
            num_items = parse(Int, num_items)
            best_known_solution = parse(Int, best_known_solution)
            index += 1
            items = []
            for _ in 1:num_items
                push!(items, parse(Float64, strip(lines[index])))
                index += 1
            end
            push!(problems, (problem_id, bin_capacity, num_items, best_known_solution, items))
        end
    end
    return problems
end

read_bin_packing_instance (generic function with 1 method)

In [3]:
function bin_packing_glpk(bin_capacity, items)
    n = length(items)
    max_bins = n  # En el peor de los casos, cada ítem puede ir en un contenedor separado
    
    model = Model(GLPK.Optimizer)
    
    # Variables
    @variable(model, x[1:max_bins, 1:n], Bin)
    @variable(model, y[1:max_bins], Bin)

    # Restricciones
    @constraint(model, [j=1:n], sum(x[i,j] for i in 1:max_bins) == 1)
    @constraint(model, [i=1:max_bins], sum(x[i,j] * items[j] for j in 1:n) <= bin_capacity * y[i])

    # Función objetivo
    @objective(model, Min, sum(y))

    optimize!(model)
    
    println("Status: ", termination_status(model))
    println("Objective value: ", objective_value(model))
    println("Número de contenedores utilizados: ", sum(value.(y)))
    
    return value.(y)
end

bin_packing_glpk (generic function with 1 method)

In [4]:
# Función para resolver el problema de bin packing con CBC
function bin_packing_cbc(bin_capacity, items)
    n = length(items)
    max_bins = n  # En el peor de los casos, cada ítem puede ir en un contenedor separado
    
    model = Model(Cbc.Optimizer)
    
    # Variables
    @variable(model, x[1:max_bins, 1:n], Bin)
    @variable(model, y[1:max_bins], Bin)

    # Restricciones
    @constraint(model, [j=1:n], sum(x[i,j] for i in 1:max_bins) == 1)
    @constraint(model, [i=1:max_bins], sum(x[i,j] * items[j] for j in 1:n) <= bin_capacity * y[i])

    # Función objetivo
    @objective(model, Min, sum(y))

    # Resolver el modelo
    optimize!(model)
    
    # Obtener número de contenedores utilizados
    num_bins_used = sum(value.(y))
    
    return num_bins_used
end


bin_packing_cbc (generic function with 1 method)

In [5]:
# Función para leer y describir todas las instancias de un archivo
function describe_problems(problems)
    for (problem_id, bin_capacity, num_items, best_known_solution, items) in problems
        println("Resolviendo problema: $problem_id, con $num_items de items, se espera $best_known_solution, $items items")
    end
    return problems
end

describe_problems (generic function with 1 method)

In [6]:
function process_and_solve__problem(problems)
    (problem_id, bin_capacity, num_items, best_known_solution, items) = problems[1]
    println("Resolviendo problema: $problem_id, con $num_items items, se espera solución óptima: $best_known_solution")
    num_bins_used = bin_packing_cbc(bin_capacity, items)
    println("Número de contenedores utilizados: $num_bins_used")
    println("Solución conocida óptima/best: $best_known_solution")
end

problems = read_bin_packing_instance("./instances/binpack5.txt")
process_and_solve__problem(problems)